<a href="https://www.kaggle.com/code/rishabhsingh18/model-ark-experiment?scriptVersionId=188203116" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import pandas as pd
import numpy as np
import torchvision.transforms as transforms
from torchvisions.datasets import ImageFolder
from torch.utils.data import DataLoader

# data preparation
def load_preprocess_data(data_path):
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    dataset = ImageFolder(root=data_path, transform-transform)
    data_loader = DataLoader(dataset, batch_size=32, shuffle=True)
    return data_loader
# insert path to the datasets in ''
data_loader = load_preprocess_data('/kaggle/input/nih-chest-x-ray-14-224x224-resized/Data_Entry_2017.csv')

# Model Implementation

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models

class StudentTeacherModel(nn.Module):
    def __init__(self, base_model):
        super(StudentTeacherModel, self).__init__()
        self.base_model = base_model
        # assuming 14 classes
        self.classification_head = nn.Linear(base_model.fc.in_features, 14)
        self.segmentation_head = nn.Conv2d(base_model.fc.in_feature, 1, kernel_size=1)
        
    def forward(self, x):
        features = self.base_model(x)
        classification_output = self.classification_head(features)
        segmentation_output = self.segmentation_head(features.unsqueeze(2).unsqueeze(3))
        return classfication_output, segmentation_output

base_model = models.resnet50(pretrained=True)
model = StudentTeacherModel(base_model)

# Training

In [ ]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training Loop
for epoch in range(10):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(data_loader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs, _ = model(inputs)
        loss = criterion (outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch {epoch+1}, Loss: {running_loss/len(data_loader)}')

# Evaluation

In [ ]:
from sklearn.metrics import accuracy_score, roc_auc_score

def evaluate_model(model, data_loader):
    model.eval()
    all_labels = []
    all_predictions = []
    with torch.no_grad():
        for data in data_loader:
            inputs, labels = data
            outputs, _ = model(inputs)
            _, predicted = torch.max(outputs, 1)
            all_labels.extend(labels.numpy())
            all_predictions.extend(predicted.numpy())
    accuracy = accuracy_score(all_labels, all_predictions)
    auc_roc = roc_auc_score(all_labels, all_predictions, multi_class='ovo')
    print(f'Accuracy: {accuracy}, AUC-ROC: {auc_roc}')
    
evaluate_model(model, data_loader)

# Analysis and Reporting via Matplotlib